<a href="https://colab.research.google.com/github/alvarito2302/visualizacion-datos/blob/main/Ejercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install plotly.express
import plotly_express as px
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
data_file_path = '/content/drive/MyDrive/Colab Notebooks/Visualizacion/tiendas_ventas_satisfaccion.csv'
data = pd.read_csv(data_file_path,sep=',')
data.head()

Mounted at /content/drive


,Tienda,Región,Ventas,Satisfacción,Empleados,%_Ecológico,Categoría_Tamaño
0,Tienda_1,Este,424.5,10.0,36,53.5,Grande
1,Tienda_2,Oeste,571.7,5.9,8,15.4,Pequeña
2,Tienda_3,Norte,807.1,6.5,15,20.3,Pequeña
3,Tienda_4,Este,547.3,6.9,21,43.5,Mediana
4,Tienda_5,Este,514.7,5.3,42,51.6,Grande


1. ¿Qué regiones tienen, en promedio, las ventas más altas?

Se ha elegido el gráfico de barras porque permite comparar de manera clara y visual el promedio de ventas entre regiones. Cada barra representa una región y su altura indica el valor medio de ventas, lo que facilita identificar que regiones tienen un rendimiento mayor y cuáles se sitúan por debajo del promedio. Este tipo de gráfico es adecuado cuando se quieren contrastar categorías discretas con valores numéricos.


In [25]:
# Calculamos promedio de ventas por región
ventas_region = data.groupby('Región', as_index=False)['Ventas'].mean().sort_values('Ventas', ascending=False)

fig = px.bar(ventas_region, x='Región', y='Ventas', color='Región',title='Promedio de Ventas por Región', orientation='v')
fig.show()

2. ¿Existe relación entre el nivel de satisfacción del cliente y las ventas?

Se ha elegido el gráfico de dispersión porque permite analizar la relación entre dos variables numéricas, en este caso, el nivel de satisfacción del cliente y las ventas. Cada punto del gráfico representa una tienda, mostrando simultáneamente su nivel de satisfacción y su volumen de ventas. Este tipo de gráfico resulta adecuado para identificar tendencias o patrones de correlación.

In [30]:
fig = px.scatter(data, x='Satisfacción', y='Ventas', color='Región', title='Relación entre Satisfacción del Cliente y Ventas')

trend = px.scatter(data, x='Satisfacción', y='Ventas', trendline='ols')
trend.data[1].line.color = 'black'
fig.add_traces(trend.data[1])

fig.show()

3.¿Cómo se distribuyen las ventas entre tiendas pequeñas medianas y grandes?

Se ha elegido el diagrama de caja (boxplot) porque permite analizar de forma clara la distribución de las ventas según el tamaño de la tienda: pequeña, mediana o grande. Este gráfico muestra la mediana, los cuartiles y los posibles valores atípicos de cada grupo, lo que facilita visualizar la dispersión y la variabilidad de las ventas. Además, permite identificar qué tipo de tienda genera en promedio más ventas y detectar diferencias significativas entre los distintos tamaños de tiendas.



In [31]:
fig = px.box(data, x='Categoría_Tamaño', y='Ventas', color='Categoría_Tamaño',title='Distribución de Ventas por Tamaño de Tienda')

fig.show()

4. ¿Qué proporción del total de ventas proviene de cada región?

Se ha elegido el gráfico de anillo porque permite mostrar de forma clara qué proporción del total de ventas corresponde a cada región. Cada segmento representa la contribución de una región respecto al total, facilitando identificar cuáles generan la mayor parte de las ventas y cuáles tienen una participación menor. Este tipo de gráfico es adecuado cuando se quiere comparar partes de un todo y entender la distribución relativa entre categorías.

In [8]:
# Calculamos ventas totales por región y ordenar de mayor a menor
ventas_region = data.groupby('Región', as_index=False)['Ventas'].sum().sort_values('Ventas', ascending=False)

fig = px.pie(ventas_region, names='Región', values='Ventas', hole=0.4,title='Proporción del Total de Ventas por Región (ordenado)',category_orders={'Región': ventas_region['Región'].tolist()})

fig.show()

5. ¿Cómo se comparan las regiones en múltiples indicadores (ventas satisfacción, % ecológico, empleados)?

Opcion a)

Se ha elegido utilizar gráficos de barras separados para cada variable porque permiten comparar de manera clara los valores de cada indicador entre regiones, sin que las diferencias de escala entre las variables afecten la visualización. Cada gráfico muestra un indicador distinto (ventas, satisfacción, % ecológico, empleados), facilitando identificar qué regiones destacan en cada uno y permitiendo una interpretación precisa de los datos. Este enfoque es adecuado cuando se quieren analizar múltiples métricas con rangos diferentes de forma individual.


In [32]:
import plotly.express as px

cols = ['Ventas', 'Satisfacción', '%_Ecológico', 'Empleados']
df_region = data.groupby('Región')[cols].mean().reset_index()

for col in cols:
    fig = px.bar(df_region, x='Región', y=col, color='Región', text=df_region[col], title=f'Promedio de {col} por Región')
    fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
    fig.show()

Opcion b)

Se ha elegido el gráfico de radar para comparar múltiples indicadores entre regiones de forma visual y proporcional. Cada eje representa un indicador distinto (ventas, satisfacción, % ecológico y empleados), y cada línea muestra cómo se desempeña una región en todos ellos. La normalización de los valores permite comparar variables con rangos distintos, mientras que la transparencia facilita observar la superposición de las regiones. Este tipo de gráfico es adecuado para identificar rápidamente fortalezas y debilidades relativas de cada región en diferentes métricas.

In [35]:
from sklearn.preprocessing import MinMaxScaler

cols = ['Ventas', 'Satisfacción', '%_Ecológico', 'Empleados']
df_region = data.groupby('Región')[cols].mean().reset_index()
df_region[cols] = MinMaxScaler().fit_transform(df_region[cols])

df_long = df_region.melt(id_vars='Región', var_name='Indicador', value_name='Valor')

fig = px.line_polar(df_long, r='Valor', theta='Indicador', color='Región',
                    line_close=True, title='Comparación de Indicadores por Región')
fig.update_traces(fill='toself', opacity=0.5)
fig.show()